In [1]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup

In [7]:
def get_url(search_item, page):
    """Search the item using the url."""

    init_url = "https://www.myntra.com/{}"
    search_item = search_item.replace(" ", "-")
    final_url = init_url.format(search_item)
    final_url = final_url + "?p={}"
    return final_url.format(page)

def extractRow(item):
    """Extract details of each row of Amazon Product."""

    try:
        URL = 'https://www.myntra.com/' + item.find('a', {'target': '_blank'})['href']
    except AttributeError:
        URL = ''

    try:
        brand = item.find('h3', {'class': 'product-brand'}).text
    except AttributeError:
        brand = ''

    try:
        product_type = item.find('h4', {'class': 'product-product'}).text
    except AttributeError:
        product_type = ''

    try:
        sizes = item.find('h4', {'class': 'product-sizes'}).text
    except AttributeError:
        sizes = ''

    try:
        product_price = item.find('div', {'class': 'product-price'})

        try:
            discounted_price = product_price.find('span', {'class': 'product-discountedPrice'}).text
        except AttributeError:
            discounted_price = ''

        try:
            striked_price = product_price.find('span', {'class': 'product-strike'}).text
        except AttributeError:
            striked_price = ''

        try:
            discount_percent = product_price.find('span', {'class': 'product-discountPercentage'}).text
        except AttributeError:
            discount_percent = ''

        if (discounted_price == '') & (striked_price == '') & (discount_percent == ''):
            product_price = product_price.text
        else:
            product_price = ''

    except AttributeError:
        product_price = ''

    return (product_type, brand, sizes, product_price, discounted_price, striked_price, discount_percent, URL)

records = list()

def main(search, pages):
    """Executes the main program."""

    # Initialize the Chrome driver
    driver = webdriver.Chrome("chromedriver")

    record = list()

    for page in range(1, pages+1):
        driver.get(get_url(search, page))
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        results = soup.findAll('li', {'class': 'product-base'})

        for result in results:
            record = extractRow(result)

            if record:
                records.append(record)

    driver.close()

    # # Saving file to a CSV

    cols = ['ProductType', 'Brand', 'Sizes', 'ProductPrice', 'DiscountedPrice', 'StrikedPrice', 'DiscountPercent', 'URL']
    data = pd.DataFrame(data=records, columns=cols)
    data.to_csv('results.csv', index=False)

In [8]:
# Search and get the data associated with the product
search = input("Enter the Product:")
main(search=search, pages=2)